In [ ]:
# !pip install -q "langgraph>=0.2.0" "langchain-openai>=0.2.0"  "langchain-community>=0.2.0" "tavily-python>=0.4.0"



In [ ]:
!pip install -U langchain langgraph langchain-cohere langchain-community


In [ ]:
# Agentic RAG (LangGraph + Command-R + Tavily)

###############################################
# 0. Kurulum Gerekli Paketler
###############################################
#!pip install langchain langgraph langchain-fireworks langchain-community

###############################################
# 1. Gerekli Modül ve Araçları Çağır
###############################################
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_fireworks import ChatFireworks
import os

###############################################
# 2. API Anahtarı (Fireworks AI)
###############################################
# Fireworks hesabından aldığın API key'i aşağıdaki gibi tanımlamalısın:
import os


# Agentic RAG: LangGraph + Cohere (Command) + Tavily
# Amaç: Sağlık konularında güvenilir kaynaklara dayalı web aramalı yanıt veren ajan

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.tools.tavily_search import TavilySearchResults

# >>> LLM: Cohere (Command) — tool calling destekli <<<
from langchain_cohere import ChatCohere


os.environ["TAVILY_API_KEY"] = "kendi apinizi yazalım"


os.environ["COHERE_API_KEY"]="kendi apinizi yazalım"

# 1) LLM tanımı (Command-A daha güncel; Command-R+ da kullanılabilir)
#   - command-a-03-2025: Cohere'ın yeni önerilen modeli (tool calling iyi)
#   - command-r-plus   : Klasik Command-R+ (tool calling mevcut)
llm = ChatCohere(
    model="command-a-03-2025",  
    temperature=0.1,
)

# 2) Web arama aracı (Tavily)
search_tool = TavilySearchResults(
    max_results=5,
    search_depth="basic",
    include_answer=True,
    include_raw_content=False,
)
tools = [search_tool]

# LLM'e tool çağırma yeteneğini bağlıyoruz
llm_with_tools = llm.bind_tools(tools)

# 3) Ajan düğümü: LLM'e mevcut mesaj geçmişini ver, gerekirse tool çağrısı isteyecek
#bu fonksiyon Grafın bir düğümü (node) olacak
def agent_node(state: MessagesState) -> MessagesState:
    messages = state["messages"]    #içinde o ana kadar biriken sohbet mesajları var.
    response = llm_with_tools.invoke(messages)  # AIMessage döner; tool_calls içerebilir
    return {"messages": messages + [response]}

# 4) Tool düğümü: LLM tool istediğinde LangGraph hazır ToolNode'u çalıştırır
tools_node = ToolNode(tools)

# 5) Koşullu geçiş: Son AI mesajında tool çağrısı var mı?
#agent düğümünden sonra hangi düğüme gideceğine karar veren router fonksiyonu.
#last.tool_calls özelliği var mı diye bakıyor. Eğer LLM, “araç çağırmalıyım” dediyse bu alan boş olmayan bir liste/dict olur.
# Eğer tool_calls varsa "tools" düğümüne geç, yoksa END’e git.

def should_continue(state: MessagesState):
    last = state["messages"][-1]
    if getattr(last, "tool_calls", None):
        return "tools"
    return END

# 6) Grafı kur
graph = StateGraph(MessagesState) #Yeni bir StateGraph oluşturuyorsun.
graph.add_node("agent", agent_node) #Biraz önce yazdığın agent_node fonksiyonunu çalıştıran node.
graph.add_node("tools", tools_node) #Tavily vb. tool’ları çalıştıran ToolNode.

graph.add_edge(START, "agent") #Grafın başlangıç kenarı: START → "agent" , Yani graf çağrıldığında ilk çalışacak düğüm agent.
graph.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END}) #"tools": "tools" : Eğer router "tools" derse "tools" node’una. END: END : Eğer router END dönerse, graf hemen sonlanır.
graph.add_edge("tools", "agent") # Tool çağrısından sonra tekrar agent düğümüne dön.

# tools düğümünden sonra tekrar "agent" düğümüne dönüyoruz.
# Döngü:
# agent LLM’i çalıştırır. Eğer tool kullanmak isterse:
# should_continue → "tools"
# tools ilgili tool’u (Tavily) çalıştırır.
# Tool çıktısı mesajlara eklenir, sonra tekrar agent’a dönülür.
# Agent bu sefer hem kullanıcı mesajını hem de tool çıktısını görerek nihai cevabı üretir (ve genelde tool_calls olmadan bitirir).

app = graph.compile() #Grafı derleyip çalıştırmaya hazır hale getiriyoruz.

# 7) Test: Sağlık senaryosu
init_messages = [
    SystemMessage(
        content=(
            "Sen bir sağlık uzmanısın. Bilgi verirken güvenilir, kanıta dayalı kaynakları kullan. "
            "Gerekirse web araması yap ve atıf/citation ekle."
        )
    ),
    HumanMessage(
        # content="Obezite ve tip 2 diyabet ilişkisini, güncel kılavuzlara referans vererek açıkla."
        content="Migren tedavisinde en güncel yaklaşımlar nelerdir?"
    ),
]

final_state = app.invoke({"messages": init_messages}) #Grafı başlatıyoruz; başlangıç durumu olarak init_messages veriyoruz.
print("\n--- AJAN CEVABI ---\n")
print(final_state["messages"][-1].content)

#BİZ BURADA LLM-tabanlı tool routing yaptık. çünkü “Tool kullanılsın mı?” sorusunu LLM cevaplıyor;

# agent → should_continue → (tools | END) akışında, son AI mesajında tool_calls varsa LangGraph tools düğümüne geçti;
# yoksa END’e gitti. Yani hangi aracın çağrılacağına LLM karar verdi, 
# LangGraph ise bu kararı koşullu kenarla (conditional edge) uyguladı.


/Users/arisoy/anaconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/97/hx1gdk894ld8_ffpqcx13mx00000gn/T/ipykernel_8622/3685141863.py:49: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(



--- AJAN CEVABI ---

Migren tedavisinde en güncel yaklaşımlar arasında ilaç kullanmadan yapılacak şeyler ve ilaç tedavileri bulunmaktadır.

**İlaçsız Tedaviler:**
- Başa buz dolu torba koymak veya soğuk suyun altına başın tutulması
- Boyna sıcak uygulamak

**İlaç Tedavileri:**
- Atak tedavisi: Ağrıyı azaltmak için analjezikler ve triptanlar kullanılır.
- Önleyici tedavi: CGRP reseptörlerini bloke eden gepant grubu yeni bir ilaçtır ve migrenin koruyucu tedavisinde etkilidir.
- Lasmiditan: Yeni geliştirilen bir ilaç grubu olan Lasmiditan, atak sırasında oldukça etkilidir ancak kalp ve beyin damar hastalığı riski bulunmaktadır.

Bu yaklaşımlar, migren tedavisinde en güncel ve etkili yöntemler olarak kabul edilmektedir. Ancak, her hastanın durumu farklı olabileceğinden, tedavi seçenekleri doktor tarafından değerlendirilmelidir.


In [2]:
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage, AIMessage

def run_health_test(user_question: str):
    init_messages = [
        SystemMessage(
            content=(
                "Sen bir sağlık uzmanısın. Bilgi verirken kanıta dayalı ol, "
                "gerektiğinde web araması yap ve kaynak/citation ekle. "
                "Tıbbi tavsiye yerine eğitim amaçlı bilgi ver."
            )
        ),
        HumanMessage(content=user_question),
    ]
    state = app.invoke({"messages": init_messages})

    # Son yanıt
    print("\n=== AJAN CEVABI ===\n")
    print(state["messages"][-1].content)

    # Araç kullanımı kontrolü (Tavily çalıştı mı?)
    tool_calls = sum(1 for m in state["messages"] if isinstance(m, ToolMessage))
    print("\n=== TEKNİK ÖZET ===")
    print(f"Tool çağrısı sayısı (Tavily): {tool_calls}")
    ai_turns = sum(1 for m in state["messages"] if isinstance(m, AIMessage))
    print(f"AI tur sayısı: {ai_turns}")

run_health_test(
    "Gebelikte hipertansiyon yönetimi için güncel kılavuzlara dayanarak "
    "ilk-basamak ilaçlar, kontrendikasyonlar ve takip önerilerini madde madde açıkla. "
    "Webden araştır ve kaynak/citation ekle."
)



=== AJAN CEVABI ===

Gebelikte hipertansiyon yönetimi, anne ve fetüs sağlığı açısından kritik öneme sahiptir. Güncel kılavuzlara dayalı olarak, ilk basamak ilaçlar, kontrendikasyonlar ve takip önerileri aşağıdaki gibidir:

### **İlk Basamak İlaçlar:**
1. **Labetalol**: Hızlı etkisi ve gebelikte güvenli olması nedeniyle ilk tercih edilen ilaçtır.
2. **Metildopa**: Gebelikte hipertansiyon tedavisinde uzun süredir kullanılan ve güvenli bulunan bir diğer seçenektir.
3. **Nifedipin**: Kısa etkili formları, gebelikte hipertansiyon kontrolünde kullanılabilir.

### **Kontrendikasyonlar:**
1. **ACE İnhibitörleri (Anjiyotensin Dönüştürücü Enzim İnhibitörleri)**: Gebeliğin ikinci ve üçüncü trimestrinde kontrendikedir, çünkü fetüs için ciddi riskler taşır.
2. **ARB'ler (Anjiyotensin Reseptör Blokerleri)**: ACE inhibitörleri gibi, gebeliğin ikinci ve üçüncü trimestrinde kontrendikedir.
3. **Tiazid Diüretikler**: Gebelikte genellikle önerilmez, çünkü elektrolit dengesizliklerine neden olabilir.

##